# YRoration (YGate) - QKeras ML Models

## Configuration

Pulse and fidelity computation are computation intensive (slow).

In [1]:
PULSE_ENABLED = False
FIDELITY_ENABLED = True

Parameters are stored in a shared script.

In [2]:
from parameters import *

Enable seeding for reproducibility of the training.

## Library

In [3]:
import os
import numpy as np
import pandas as pd

import sys
sys.path.append('..')

from utils.helpers import *

# Disable some console warnings
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## Check GPU

## Dataset

### Load

In [4]:
x_train = np.load('./data/yrotation/{}/x_train.npy'.format(CSV_CONFIG))
x_val = np.load('./data/yrotation/{}/x_val.npy'.format(CSV_CONFIG))
x_test = np.load('./data/yrotation/{}/x_test.npy'.format(CSV_CONFIG))
y_train = np.load('./data/yrotation/{}/y_train.npy'.format(CSV_CONFIG))
y_val = np.load('./data/yrotation/{}/y_val.npy'.format(CSV_CONFIG))
y_test = np.load('./data/yrotation/{}/y_test.npy'.format(CSV_CONFIG))

In [5]:
N_ANGLES = x_train.shape[1]
N_PARAMS = y_train.shape[1]

In [6]:
print('---------------------------------------')
print('- Pulse parameters (y) #', N_PARAMS) # ASSUMING 1 BETA VALUE (ANGLE ALONG Y AXIS)
print('- Gate parameters  (x) #', N_ANGLES) # ASSUMING 1 BETA VALUE (ANGLES ALONG Y AXIS)
print('---------------------------------------')

---------------------------------------
- Pulse parameters (y) # 20
- Gate parameters  (x) # 1
---------------------------------------


### Evaluation

<span style="color:red">ATTENTION: Measuring fidelity on the entire train, validation, and test sets could be time consuming. Use `limit` parameter in case.</span>

## MLP

### Load Keras Model

In [7]:
import tensorflow as tf

best_model_file = PREFIX + '/best_keras_model.h5'
def sine_activation(x):
    return tf.math.sin(x)
model = tf.keras.models.load_model(best_model_file, custom_objects={'sine_activation': sine_activation})
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.0001),
    loss='mean_squared_error',
    metrics=['mean_squared_error'])

In [8]:
model_id = MODEL_ID_PREFIX + get_basic_id(model)
print(model_id)

smallMLP_1x12x8x16x12x8x20


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc0 (Dense)                  (None, 12)                24        
_________________________________________________________________
sine0 (Activation)           (None, 12)                0         
_________________________________________________________________
fc1 (Dense)                  (None, 8)                 104       
_________________________________________________________________
sine1 (Activation)           (None, 8)                 0         
_________________________________________________________________
fc2 (Dense)                  (None, 16)                144       
_________________________________________________________________
sine2 (Activation)           (None, 16)                0         
_________________________________________________________________
fc3 (Dense)                  (None, 12)                2

In [49]:
#mse, msle, mape = model.evaluate(x_test, y_test)
mse = model.evaluate(x_test, y_test)[0]
msle, mape = None, None

512/512 [==============================] - 0s 343us/step - loss: 1.6350e-07 - mean_squared_error: 1.6350e-07


In [50]:
mse = model.evaluate(x_val, y_val)[0]
msle, mape = None, None

128/128 [==============================] - 0s 358us/step - loss: 1.9285e-07 - mean_squared_error: 1.9285e-07


#### QAT/QKeras

##### Create model

A classical multi-layer perceptron: 3 (inputs), 8 hidden layers w/ 128 neurons, and 20 (outputs).

<span style="background-color:orange">This may be oversized for the final hardware implementation, but it is a starting point. The bit-precision is likely oversized too. We can use AutoQKeras or manually tune the hyper-parameters.</span>

In [23]:
import tensorflow as tf
import qkeras
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu
from tensorflow.keras.layers import Layer

Remember that `ap_fixed<16, 8, true>` is `quantized_bits(bits=16,integer=7)`.

In [41]:
# Build quantized model
W=32
I=16
QN=0
A=1

qmodel = tf.keras.models.Sequential()
#qmodel.add(Input(shape=(1,), name='input1'))
qmodel.add(QDense(NEURONS_PER_LAYER[0],
                  input_shape=(1,),
                  name='fc0',
                  kernel_quantizer=quantized_bits(bits=W,integer=I-1,alpha=A,qnoise_factor=QN),
                  bias_quantizer=quantized_bits(bits=W,integer=I-1,alpha=A,qnoise_factor=QN),
                  kernel_initializer='lecun_uniform'))
qmodel.add(QActivation(activation=quantized_relu(bits=W,integer=I-1,qnoise_factor=QN),
                       name='relu0'))

for i, n in enumerate(NEURONS_PER_LAYER[1:]):
    qmodel.add(QDense(n,
                  name='fc{}'.format(i+1),
                  kernel_quantizer=quantized_bits(bits=W,integer=I-1,alpha=A,qnoise_factor=QN),
                  bias_quantizer=quantized_bits(bits=W,integer=I-1,alpha=A,qnoise_factor=QN),
                  kernel_initializer='lecun_uniform'))
    qmodel.add(QActivation(activation=quantized_relu(bits=W,integer=I-1,qnoise_factor=QN),
                       name='relu{}'.format(i+1)))
qmodel.add(QDense(N_PARAMS,
                  name='fc{}'.format(len(NEURONS_PER_LAYER)),
                  kernel_quantizer=quantized_bits(bits=W,integer=I-1,alpha=A,qnoise_factor=QN),
                  bias_quantizer=quantized_bits(bits=W,integer=I-1,alpha=A,qnoise_factor=QN),
                  kernel_initializer='lecun_uniform'))

In [42]:
model_id = MODEL_ID_PREFIX + get_basic_id(qmodel)
print(model_id)

smallMLP_1x12x8x16x12x8x20


In [43]:
qmodel.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc0 (QDense)                 (None, 12)                24        
_________________________________________________________________
relu0 (QActivation)          (None, 12)                0         
_________________________________________________________________
fc1 (QDense)                 (None, 8)                 104       
_________________________________________________________________
relu1 (QActivation)          (None, 8)                 0         
_________________________________________________________________
fc2 (QDense)                 (None, 16)                144       
_________________________________________________________________
relu2 (QActivation)          (None, 16)                0         
_________________________________________________________________
fc3 (QDense)                 (None, 12)               

##### Training

In [44]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [45]:
best_model_file = PREFIX + '/best_qkeras_model.h5'
last_model_file = PREFIX + '/last_qkeras_model.h5'

Enable training (`train_and_save`) to overwrite the model file.

In [46]:
train_and_save = True

Set weights from the Keras model into the QKeras model

In [47]:
qmodel.set_weights(model.get_weights())
qmodel.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.0001),
    loss='mean_squared_error',
    metrics=['mean_squared_error'])
qmodel.save(best_model_file)

In [56]:
# Assuming `model` is your Keras model and `qmodel` is your QKeras model
for i in range(len(model.layers)):
    keras_weights = model.layers[i].get_weights()
    qkeras_weights = qmodel.layers[i].get_weights()
    for kw, qw in zip(keras_weights, qkeras_weights):
        print('Layer', i)
        print('Keras weights:')
        print(kw)
        print('QKeras weights:')
        print(qw)
        print('----------------------------------------')


Layer 0
Keras weights:
[[ 0.92229646  1.4113047   0.34144127 -1.1159358  -0.52307355  1.4264936
   1.5917554   0.7760752  -1.2379053  -0.06260239  1.6291194  -0.62103033]]
QKeras weights:
[[ 0.92229646  1.4113047   0.34144127 -1.1159358  -0.52307355  1.4264936
   1.5917554   0.7760752  -1.2379053  -0.06260239  1.6291194  -0.62103033]]
----------------------------------------
Layer 0
Keras weights:
[ 0.00370913  0.01557205 -0.00545642 -0.00074088 -0.00037202  0.00951494
  0.00498495  0.0002311  -0.0096365   0.00255276  0.00814393  0.00247453]
QKeras weights:
[ 0.00370913  0.01557205 -0.00545642 -0.00074088 -0.00037202  0.00951494
  0.00498495  0.0002311  -0.0096365   0.00255276  0.00814393  0.00247453]
----------------------------------------
Layer 2
Keras weights:
[[-0.41244256  0.07581601 -0.451056    0.28145915 -0.20098367 -0.06725013
   0.4688105  -0.24980712]
 [ 0.2791882   0.03889751 -0.3155419  -0.02344753  0.5636125   0.30340692
   0.24396378  0.05806979]
 [-0.4171369   0.094940

##### Simply porting model gives really bad models for some reason so fine tune by training again

We use Adam optimizer, minimize the Mean Squared Error, and early stop.

In [58]:

history = None
if train_and_save:
    qmodel.compile(
        optimizer=tf.optimizers.Adam(learning_rate=0.0001),
        loss='mean_squared_error',
        metrics=[
            'mean_squared_error'
        ])
    early_stopping = EarlyStopping(
        monitor='val_loss',
        mode='min',
        patience=5000,
        verbose=1)
    model_checkpoint = ModelCheckpoint(
        filepath=best_model_file,
        monitor='val_loss',
        mode='min',
        save_best_only=True)
    history = qmodel.fit(
        x_train,
        y_train,
        epochs=5000,
        batch_size=128, # default 32
        validation_data=(x_val, y_val),
        callbacks=[early_stopping, model_checkpoint],
        verbose=0)
    qmodel.save(last_model_file)

Load the saved best model and use it from now on.

In [59]:
co = {}
qkeras.utils._add_supported_quantized_objects(co)
qmodel = tf.keras.models.load_model(best_model_file, custom_objects=co)

##### Evaluation

Although we may plot and print many metrics, we focus only on **Mean Squared Error (MSE).**

Plot training history. (Not needed if no retraining of QKeras model is done)

Print metrics.

In [60]:
#qmse, qmsle, qmape = qmodel.evaluate(x_test, y_test)
qmse = qmodel.evaluate(x_test, y_test)[0]
qmsle, qmape = None, None

512/512 [==============================] - 0s 397us/step - loss: 1.1710e-07 - mean_squared_error: 1.1710e-07


In [61]:
y_qkeras = qmodel.predict(x_test)

In [62]:
import random
hash_id = random.getrandbits(32)

if FIDELITY_ENABLED:
    ygate_fidelity_qkeras = get_ygate_fidelity(x_test,
                                               y_qkeras,
                                               config_template=CONFIG_TEMPLATE_JSON,
                                               pulse_data_path='/tmp/ygate_{:x}.csv'.format(hash_id),
                                               output_objf_path='/tmp/ygate_fidelity_{:x}.csv'.format(hash_id))
else:
    ygate_fidelity_qkeras = None

In [63]:
with pd.option_context('display.float_format', '{:0.12f}'.format):
    data = pd.DataFrame([["QKeras", qmse, qmsle, qmape, ygate_fidelity_qkeras]], columns=["", "MSE", "MSLE", "MAPE", "Fidelity"])
    display(data)

,,MSE,MSLE,MAPE,Fidelity
0,QKeras,0.000000117102,None,None,0.999976231530


Save metrics values to file for future reference.

In [64]:
metrics_filename = PREFIX + '/metrics.csv'

# You can disable the writing if necessary.
write_metrics_csv = True

In [65]:
from utils.metrics import write_metrics
if write_metrics_csv:
    write_metrics(metrics_filename, 'QKeras', model_id, qmse, qmsle, qmape, ygate_fidelity_qkeras)

In [66]:
from utils.metrics import print_metrics
print_metrics(metrics_filename)

,Date,Framework,ID,MSE,MSLE,MAPE,Fidelity
0,04/06/2023 23:32:36,Keras,smallMLP_1x12x8x16x12x8x20,0.000000707116,NaN,NaN,0.990096302650
1,04/06/2023 23:41:26,Keras,smallMLP_1x16x16x16x16x16x16x16x20,0.000000302866,NaN,NaN,0.996592127711
2,05/06/2023 00:54:06,Keras,smallMLP_1x16x16x16x16x16x16x16x20,0.000000398108,NaN,NaN,0.995077184167
3,05/06/2023 01:35:19,Keras,smallMLP_1x16x16x16x16x16x16x16x20,0.000000326932,NaN,NaN,0.995693778354
4,05/06/2023 17:02:19,Keras,smallMLP_1x16x16x16x16x16x16x16x20,0.000003161936,NaN,NaN,0.978272227910
5,05/06/2023 17:41:50,Keras,smallMLP_1x16x16x16x16x16x16x16x20,0.000000187962,NaN,NaN,0.999987138670
6,05/06/2023 22:18:00,QKeras,smallMLP_1x16x16x16x16x16x16x16x20,0.001994839637,NaN,NaN,0.360618878665
7,05/06/2023 22:26:55,Keras,smallMLP_1x12x8x16x12x8x20,0.000000192853,NaN,NaN,0.999983054298
8,05/06/2023 22:29:33,QKeras,smallMLP_1x12x8x16x12x8x20,0.078390441835,NaN,NaN,0.252139220410
9,05/06/2023 23:26:05,QKeras,smallMLP_1x12x8x16x12x8x20,0.000000138960,NaN,NaN,0.999987939607
